In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Import all files to notebook

In [2]:
train = pd.read_csv(r'..\data\house prediction\train.csv',index_col = 'Id')
test = pd.read_csv(r'..\data\house prediction\test.csv',index_col = 'Id')
ans = pd.read_csv(r'..\data\house prediction\sample_submission.csv',index_col = 'Id')

In [ ]:
# test = pd.merge(test,ans,on= 'Id')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-

In [ ]:
# null = []
# for i in train.columns:
#     val = pd.value_counts(train[i].isna())
#     if val[0]/len(train) <= 0.6:
#         null.append(i)

# print("The columns must delete are : ",null)

In [ ]:
# train_delete_col = train.drop(columns=null)
# test_delete_col = test.drop(columns=null)

In [ ]:
# train_clean = train_delete_col.fillna(np.median)
# test_clean = test_delete_col.fillna(np.median)
# train_clean = train_delete_col.fillna(train_delete_col.median())
# train_clean = train_clean.dropna(axis = 0)
# test_clean = test_delete_col.fillna(train_delete_col.median(),inplace = True)

In [4]:
def feature_select(data):
    null = []
    for i in train.columns:
        val = pd.value_counts(train[i].isna())
        if val[0]/len(train) <= 0.6:
            null.append(i)
    data.drop(columns=null,inplace = True)
    data.fillna(data.median(),inplace = True)
    data.dropna(axis = 0,inplace = True)
    return data

In [5]:
train_clean = feature_select(train)

In [ ]:
object_col = train_clean.select_dtypes(include='object').columns
object_index = train_clean.select_dtypes(include='object').index

In [ ]:
object_index

In [6]:
from sklearn.preprocessing import LabelEncoder
def encoder(data):
    encoder_data = []
    le = LabelEncoder()
    for i in data.columns:
        encoder_data.append(le.fit_transform(data[i]))
    return np.transpose(encoder_data)
#     return pd.DataFrame(np.reshape(np.array(encoder_data),(row,columns)),columns=data_col,index=data_index) 

In [10]:
def combine(data):
    col = data.columns
    ind = data.index
    data_encoder = pd.DataFrame(encoder(data[col]),columns=col,index=ind)
    return pd.merge(data.drop(columns = col),data_encoder,how='inner',on = 'Id')

In [11]:
train_complete = combine(train_clean)

In [13]:
train_complete.head(3)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,5,3,35,287,1,3,3,0,4,0,...,0,0,0,0,0,1,2,8,4,371
2,0,3,50,448,1,3,3,0,2,0,...,0,0,0,0,0,4,1,8,4,303
3,5,3,38,641,1,0,3,0,4,0,...,0,0,0,0,0,8,2,8,4,400


In [ ]:
from sklear.pra